In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
import pandas as pd
df = pd.read_csv('enriched_environmental_data_final.csv')
df.head()

In [3]:

df = df[df['Purpose'] != "בנייה נמוכה/צמודת קרקע -לייזמים/קבלנים"]
df = df.drop(columns=['Tender Number', 'Purpose','Publication date','Status', 'Winner Name', 'Submission deadline', 'Plan','Valuation Price', 'Minimum Price', 'Development Cost', 'Compound Number', 'Opening date','X','Y'])

In [4]:

# Remove missing values

print("missing data:")
print(df.isnull().sum())

df = df[df['Final Price'].notnull()]
df = df[df['Region'].notnull()]
df = df[df['Final Price'] != "אין הצעות למתחם זה"]

numeric_columns = ['Final Price']

for col in numeric_columns:
    df[col] = df[col].astype(str).str.replace(',', '').astype(float)

# Handling missing values in numerics columns
df['Units'] = df['Units'].fillna(df['Units'].median())

df['Parcel'] = df['Parcel'].fillna("Not existing")
df['Block'] = df['Block'].fillna("Not existing")
df['Section'] = df['Section'].fillna("Not existing") # remove this

missing data:
Region                                396
City                                  396
Committee date                        396
Units                                 160
Final Price                          3330
Area m2                                 0
Parcel                                595
Block                                   0
Section                                 0
dist_to_kindergarten                    0
is_kindergarten_within_radius           0
dist_to_school                          0
is_school_within_radius                 0
dist_to_university                      0
is_university_within_radius             0
dist_to_bus_stop                        0
is_bus_stop_within_radius               0
dist_to_train_station                   0
is_train_station_within_radius          0
dist_to_park                            0
is_park_within_radius                   0
dist_to_mall                            0
is_mall_within_radius                   0
dist_to_supermarket 

In [5]:

df['Final Price'] = df['Final Price'].replace(',', '', regex=True).astype(float)
df['Area m2'] = df['Area m2'].replace(',', '', regex=True).astype(float)
print(f"before filter Final Price: {len(df)}")

# Manual binderies
lower_bound = df['Final Price'].quantile(0.05)
upper_bound = df['Final Price'].quantile(0.95)

df = df[(df['Final Price'] >= lower_bound) & (df['Final Price'] <= upper_bound)]
print(f"after filter Final Price: {len(df)}")
df = df[(df['Area m2'] >= 100) & (df['Area m2'] <= 1000)]
print(f"after filter Area m2: {len(df)}")

before filter Final Price: 6900
after filter Final Price: 6210
after filter Area m2: 5962


In [6]:
df['Committee date'] = pd.to_datetime(df['Committee date'], errors='coerce')
df['Committee_year'] = df['Committee date'].dt.year
df['Committee_month'] = df['Committee date'].dt.month

df = df.drop(columns=['Committee date'])

In [7]:

print(df['Final Price'].describe())

count    5.962000e+03
mean     5.941256e+05
std      4.999464e+05
min      4.051800e+04
25%      2.478390e+05
50%      4.492025e+05
75%      7.949042e+05
max      2.428000e+06
Name: Final Price, dtype: float64


In [8]:

df['Price per m2'] = df['Final Price'] / df['Area m2']

# Saving min- max vals of price.
price_per_m2_min = df['Price per m2'].min()
price_per_m2_max = df['Price per m2'].max()

In [9]:
import json
price_minmax = {
    "min": float(price_per_m2_min),
    "max": float(price_per_m2_max)
}

with open("price_minmax.json", "w", encoding="utf-8") as f:
    json.dump(price_minmax, f, ensure_ascii=False, indent=2)

In [10]:
numeric_columns = ['Units','Final Price','Area m2','Price per m2','dist_to_kindergarten','dist_to_school','dist_to_university',
    'dist_to_bus_stop','dist_to_train_station','dist_to_park',
    'dist_to_mall','dist_to_supermarket','dist_to_beach',
    'dist_to_place_of_worship']

scaler_dict = {}

for col in numeric_columns:
    if col not in df.columns:
        print(f"Warning: Column '{col}' not found, skipping.")
        continue

    col_min = df[col].min()
    col_max = df[col].max()
    if col_min == col_max:
        print(f"Warning: column '{col}' has the same min ({col_min}) and max. Setting all scaled to 0.")
        df[col] = 0.0
    else:
        df[col] = (df[col] - col_min) / (col_max - col_min)

    scaler_dict[col] = {
        "min": float(col_min),
        "max": float(col_max)
    }

block_one_hot = pd.get_dummies(df['Block'], prefix='Block')
# section_one_hot = pd.get_dummies(df['Section'], prefix='Section')
# parcel_one_hot = pd.get_dummies(df['Parcel'], prefix='Parcel')
committee_month_one_hot = pd.get_dummies(df['Committee_month'], prefix='Committee_month')
committee_year_one_hot = pd.get_dummies(df['Committee_year'], prefix='Committee_year')


df = df.drop(['Block', 'Section', 'Parcel', 'Committee_month', 'Committee_year'], axis=1)

# df = pd.concat([df, block_one_hot, section_one_hot, parcel_one_hot, committee_month_one_hot, committee_year_one_hot], axis=1)
df = pd.concat([df, block_one_hot, committee_month_one_hot, committee_year_one_hot], axis=1)

with open("scaler_minmax.json", "w", encoding="utf-8") as f:
    json.dump(scaler_dict, f, ensure_ascii=False, indent=2)

normalized_columns = ['Final Price', 'Area m2', 'Units','dist_to_kindergarten', 'dist_to_school', 'dist_to_university',
    'dist_to_bus_stop', 'dist_to_train_station', 'dist_to_park',
    'dist_to_mall', 'dist_to_supermarket', 'dist_to_beach',
    'dist_to_place_of_worship']

In [11]:
df = pd.get_dummies(df, columns=['Region'], prefix='Region')

city_mapping = {city: idx for idx, city in enumerate(df['City'].unique())}
df['City_Index'] = df['City'].map(city_mapping)

with open("city_mapping.json", "w", encoding="utf-8") as f:
    json.dump(city_mapping, f, ensure_ascii=False, indent=2)
df = df.replace({True: 1, False: 0})

index_to_city = {int(idx): city for city, idx in city_mapping.items()}
with open("index_to_city.json", "w", encoding="utf-8") as f:
    json.dump(index_to_city, f, ensure_ascii=False)

C:\Users\idanb\AppData\Local\Temp\ipykernel_27384\2002307546.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})


In [12]:
df = df[~df['City'].str.contains("באר שבע , כלניות", na=False)]

df=df.drop(['Final Price'],axis=1)

In [13]:
df.to_csv("cleaned_data.csv", index=False)

# files.download("/content/cleaned_data.csv")
# files.download("/content/scaler_minmax.json")